# Pytorch Basic

In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device("cuda:0")   # NVIDIA GPU
elif torch.backends.mps.is_available():
    device = torch.device("mps")      # Apple Silicon GPU (M1/M2/M3)
else:
    device = torch.device("cpu")      # CPU only

print("Using device:", device)

Using device: mps


### torch.tensor 함수

`tensor(data, dtype=None, device=None, requires_grad=False) -> Tensor`

     data (array_like): list, tuple, numpy ndarray, scalar, and other types.
     dtype :  `None`인 경우 `data`에서 데이터 유형을 유추합니다.  
     device : `cpu`, `cuda`  
     require_grad(bool, optional): autograd가 작업을 기록해야 하는 경우

Float torch tensor 생성

In [6]:
# numpy 생성
a = np.ones((2, 3), dtype='float32')

# tensor 생성
b = torch.tensor(a)

# dtype
print(a.dtype, '\t', b.dtype)

float32 	 torch.float32


Integer tensor 생성

In [28]:
x = torch.tensor([[1, 2, 3], [4, 5, 6]], dtype=torch.int32)
x

tensor([[1, 2, 3],
        [4, 5, 6]], dtype=torch.int32)

### Tensor 생성

- 무작위로 초기화된 행렬 생성 (uniform distribution)

In [29]:
x = torch.rand(5, 3)
x

tensor([[0.7302, 0.5254, 0.8463],
        [0.4565, 0.1540, 0.5775],
        [0.1878, 0.6118, 0.8028],
        [0.9635, 0.6758, 0.6244],
        [0.4474, 0.8259, 0.1196]])

## Tensor 의 shape & dimension (rank)

In [21]:
a = torch.tensor([0, 1, 2, 3, 4])
a.size()

torch.Size([5])

In [22]:
# size() 와 shape 은 alias
a.shape

torch.Size([5])

In [23]:
a.ndimension()

1

## numpy 와 tensor 간의 호환성

- memory 를 공유하므로 하나를 수정하면 나머지에 모두 반영  

In [27]:
numpy_array = np.array([0.0, 1.0, 2.0, 3.0, 4.0, 5.0])
numpy_array

array([0., 1., 2., 3., 4., 5.])

In [30]:
torch_tensor = torch.from_numpy(numpy_array)
torch_tensor

tensor([0., 1., 2., 3., 4., 5.], dtype=torch.float64)

In [31]:
back_to_numpy = torch_tensor.numpy()
back_to_numpy

array([0., 1., 2., 3., 4., 5.])

In [34]:
back_to_numpy[0] = 100

In [33]:

numpy_array, torch_tensor, back_to_numpy

(array([100.,   1.,   2.,   3.,   4.,   5.]),
 tensor([100.,   1.,   2.,   3.,   4.,   5.], dtype=torch.float64),
 array([100.,   1.,   2.,   3.,   4.,   5.]))

## Scalar value

- 만약 tensor에 하나의 값만 존재한다면, ``.item()`` method를 사용하여 숫자 값을 얻을 수 있습니다.  

In [37]:
x = torch.randn(1)
x.item()

-0.859268069267273

In [39]:
a = torch.tensor([5., 3., 4., 1.])
print(a[0])
print(a[0].item())

tensor(5.)
5.0


- tensor 가 array 형태인 경우 `numpy()` method 를 통해 ndarray 반환

In [42]:
x = torch.rand(2)
print(x)
print()
print(x.numpy())

tensor([0.8728, 0.6550])

[0.872791  0.6550264]


## Basic Operations - numpy 와 동일

In [43]:
# element-wise 덧셈, 뺄셈
u = torch.tensor([1.0, 0.0])
v = torch.tensor([0.0, 1.0])
z = u + v
z

tensor([1., 1.])

In [44]:
z = u - v
z

tensor([ 1., -1.])

In [45]:
# element-wise 곱셈
u = torch.tensor([1, 2])
v = torch.tensor([3, 2])
z = u * v
z

tensor([3, 4])

In [46]:
# 스칼라 곱
y = torch.tensor([1, 2])
z = 2 * y
z

tensor([2, 4])

In [48]:
# dot product
u = torch.tensor([1, 2])
v = torch.tensor([3, 1])
result = torch.dot(u, v)
result

tensor(5)

## TORCH.AUTOGRAD를 이용한 자동 미분

- autograd 패키지는 Tensor의 모든 연산에 대해 자동 미분을 제공   

- 신경망을 훈련할 때 가장 자주 사용되는 알고리즘은 역전파이다. 이 알고리즘에서 매개 변수 (모델 가중치)는 주어진 매개 변수에 대한 손실 함수의 기울기에 따라 조정된다.  

- 이러한 그래디언트를 계산하기 위해 PyTorch에는 torch.autograd라는 내장 미분 엔진이 있다.

- ``.requires_grad`` 속성을 True 로 설정하면, 그 tensor에서 이뤄진 모든 연산들을 추적(track)하기 시작  

- 계산이 완료된 후 ``.backward()`` 를 호출하여 모든 변화도(gradient)를 자동으로 계산  

- 이 Tensor의 변화도는 ``.grad`` 속성에 누적  

- Tensor가 기록을 추적하는 것을 중단하게 하려면, ``.detach()`` 를 호출하여 연산 기록으로부터 분리(detach)하여 이후 연산들이 추적되는 것을 방지한다.

- 도함수를 계산하기 위해서는 Tensor 의 ``.backward()`` 를 호출

In [61]:
x = torch.tensor(2.0, requires_grad=True)
x

tensor(2., requires_grad=True)

In [62]:
y = x ** 2
y

tensor(4., grad_fn=<PowBackward0>)

y 를 x 에 대하여 미분  
$$y(x)=x^2$$

$$
\frac{dy}{dx}\Big|_{x=2} = 2x = 4 \;\;\; \rightarrow \;\; \text{y.backward()}
$$

In [63]:
y.backward()
x.grad

tensor(4.)